<a href="https://jupyter.org/try">
        <img src="https://img.shields.io/badge/Made%20with-Jupyter-orange?style=for-the-badge&logo=Jupyter" align="left"/>
</a><br><br>
<a href="https://cloud.ibm.com/services">
    <img src="https://img.shields.io/badge/platform-IBM%20Watson%20Studio-navy.svg" align="left"/>
</a>
<a>
    <img src="https://img.shields.io/badge/python-3.7-blue.svg" align="left"/>
</a>
<a>
    <img src="https://img.shields.io/badge/Status-Inprogress-green.svg" align="left"/>
</a>


# Applied Data Science Capstone

This is repo use for course "Applied Data Science Capstone" on **[Coursera](https://www.coursera.org/)**

    
## Infomation 

|Info| Detail|
|:------|:------|
|Owner| Truong Le|
|Start day| Mar 21, 2021|
|Status| Inprogress|
|Course link| [here](https://www.coursera.org/learn/applied-data-science-capstone?specialization=ibm-data-science)|

In [1]:
# Import library
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


# Project for week 03

## ***Prepare data - Create data frame***

In [3]:
# Install package
# !conda install -c conda-forge geopy --yes
# !conda install -c conda-forge folium=0.5.0 --yes
# !pip install geocoder

In [4]:
import xml
import json 
import folium
import requests 
import numpy as np 
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as colors
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
# Raw data by BeautifulSoup
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url,'lxml')

In [6]:
# Get value for data frame and filter data
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
        
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [7]:
df.describe()

,PostalCode,Borough,Neighborhood
count,103,103,103
unique,103,15,103
top,M5W,North York,"Clairville, Humberwood, Woodbine Downs, West H..."
freq,1,24,1


In [8]:
df.shape

(103, 3)

## ***Get latitude and longitude coordinates of each neighborhood***

In [9]:
# down load data longtitude , latitude
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv

--2021-04-18 05:09:34--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2788 (2.7K) [text/csv]
Saving to: ‘Geospatial_Coordinates.csv.1’

Geospatial_Coordina 100%[===================>]   2.72K  --.-KB/s    in 0s      

2021-04-18 05:09:34 (69.5 MB/s) - ‘Geospatial_Coordinates.csv.1’ saved [2788/2788]



In [10]:
df_latLang = pd.read_csv("Geospatial_Coordinates.csv")
df_latLang.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [11]:
df_merge = pd.merge(left=df, right=df_latLang, how='left', left_on='PostalCode', right_on='Postal Code')
df_merge.drop('Postal Code', axis=1, inplace=True)
df_merge.head(20)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
